In [2]:
import sys
import os
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

print(f"working directory: {os.getcwd()}")



from data_loading import load_data
from config_and_logging import load_config, generate_run_id, save_run_metadata, create_output_dir, log_to_mlflow
from model_pipeline import choose_best_model

import joblib

config_path = os.path.join('Configs', 'shallow4.yaml')

config = load_config(config_path=config_path)
run_name = config['run']['run_name']
run_id = generate_run_id(config)
output_dir = create_output_dir(run_name, run_id)

target_column = config['data']['target_column']
print("Run name:", run_name)
print("Run ID:", run_id)
print("Data laden...")
train_df, test_df, _ = load_data(config)


working directory: c:\Users\lackerman008\OneDrive - pwc\Outside\Code\Machine learning\Electricity Shortfall Challenge
Run name: shallow4
Run ID: _20250722_125215
Data laden...


In [1]:
from electricity_forecast.data_loading import test_training_data
import pandas as pd

bad_data = pd.DataFrame({
            'feature1': [None, None, None],
            'target': [1, 0, None]
        })

test_training_data(bad_data)

⚠️ Ignoring unexpected fields: {'target', 'feature1'}
❌ Error during data testing: 47 validation errors for PredictionRequest
features.time
  Field required [type=missing, input_value={'feature1': None, 'target': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
features.Madrid_wind_speed
  Field required [type=missing, input_value={'feature1': None, 'target': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
features.Valencia_wind_deg
  Field required [type=missing, input_value={'feature1': None, 'target': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
features.Bilbao_wind_speed
  Field required [type=missing, input_value={'feature1': None, 'target': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
features.Barcelona_wind_speed
  Field required [type=missing, input_value={'feature1': 

In [4]:
from tests.test_pipeline import TestFullPipeline

ModuleNotFoundError: No module named 'src'

In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from preprocessing import create_preprocessing_pipeline, StandardTransformerWrapper, TimeAwareKNNImputer
import pandas as pd

imputer = TimeAwareKNNImputer()

df = pd.DataFrame({
   'temperature': [20, 21, 19, 22],
   'humidity': [30, 35, 40, 45],
   'time': pd.date_range("2023-01-01", periods=4, freq="h")
})

pipeline_no_dummies = StandardTransformerWrapper(create_preprocessing_pipeline(imputer=imputer, add_time_dummies=None))
output1 = pipeline_no_dummies.fit_transform(df)

pipeline_with_dummies = StandardTransformerWrapper(create_preprocessing_pipeline(imputer=imputer, add_time_dummies='cyclical'))
output2 = pipeline_with_dummies.fit_transform(df)

print(output1.shape)
print(output2.shape)
print(pd.DataFrame(output2).head())



(2, 2)
(2, 11)
     0     1    2    3    4         5         6         7        8    9   \
0  20.0  30.0  0.0  6.0  1.0  0.000000  1.000000 -0.781831  0.62349  0.5   
1  22.0  45.0  3.0  6.0  1.0  0.707107  0.707107 -0.781831  0.62349  0.5   

         10  
0  0.866025  
1  0.866025  


In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from plots import plot_predictions
from models import train_ar_diff_model, predict_ar_diff, get_model
from preprocessing import get_pipeline_for_model, create_preprocessing_pipeline, get_imputer
from model_pipeline import split_data

print("Choose best models...")

train_val_split = config['preprocessing']['train_val_split']

best_rmse = float("inf")
best_model = None
best_model_name = ""
best_X_train = None
best_pipeline = None

y_train = train_df[config['data']['target_column']]
X_train = train_df.drop(columns=[config['data']['target_column']])

def evaluate_model(y_true, y_pred):
    """
    Bereken de Root Mean Squared Error
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))

def get_pipeline_for_model(model, config):

    if model["type"] in ["AR", "MA", "SMA"]:
        return None  # TODO pipeline to be added
    
    elif model.get('scaling', None):
        return create_preprocessing_pipeline(imputer=get_imputer(config), 
                                             freq=config['preprocessing']['freq'],
                                             fill_method=config['preprocessing']['fill_method'], 
                                             add_time_dummies=config['preprocessing']['add_time_dummies'], 
                                             scaling=model['scaling'])

    else:
        raise ValueError(f"No preprocessing pipeline defined for model_type: {model['type']}")




Choose best models...


In [59]:
print(config['models'])

[{'type': 'LinearRegression', 'params': {}, 'scaling': True}, {'type': 'RandomForest', 'params': {'n_estimators': 10, 'random_state': 42}, 'scaling': False}, {'type': 'Ridge', 'params': {'alpha': 10.0, 'random_state': 42}, 'scaling': True}, {'type': 'Lasso', 'params': {'alpha': 1.0, 'random_state': 42}, 'scaling': True}, {'type': 'ElasticNet', 'params': {'alpha': 1.0, 'l1_ratio': 0.5, 'random_state': 42}, 'scaling': True}, {'type': 'RandomForest', 'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}, 'scaling': False}, {'type': 'ExtraTreesRegressor', 'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}, 'scaling': False}, {'type': 'GradientBoostingRegressor', 'params': {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42}, 'scaling': False}, {'type': 'XGBRegressor', 'params': {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42}, 'scaling': False}, {'type': 'KNeighborsRegressor', 'params': {'n_neighbors

In [60]:
for model in config['models']:
    print(model)
    print(model["type"])

{'type': 'LinearRegression', 'params': {}, 'scaling': True}
LinearRegression
{'type': 'RandomForest', 'params': {'n_estimators': 10, 'random_state': 42}, 'scaling': False}
RandomForest
{'type': 'Ridge', 'params': {'alpha': 10.0, 'random_state': 42}, 'scaling': True}
Ridge
{'type': 'Lasso', 'params': {'alpha': 1.0, 'random_state': 42}, 'scaling': True}
Lasso
{'type': 'ElasticNet', 'params': {'alpha': 1.0, 'l1_ratio': 0.5, 'random_state': 42}, 'scaling': True}
ElasticNet
{'type': 'RandomForest', 'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}, 'scaling': False}
RandomForest
{'type': 'ExtraTreesRegressor', 'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}, 'scaling': False}
ExtraTreesRegressor
{'type': 'GradientBoostingRegressor', 'params': {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42}, 'scaling': False}
GradientBoostingRegressor
{'type': 'XGBRegressor', 'params': {'n_estimators': 100, 'max_depth': 4, 'learning_rate

In [63]:
from preprocessing import get_pipeline_for_model

for model in config['models']:
        print(model['type'])
        pipeline = get_pipeline_for_model(model, config)
        
        X_train_processed, y_train_processed = pipeline.fit_transform(X_train, y_train)

        X_train_new, X_val, y_train_new, y_val = split_data(X_train_processed, y_train_processed, train_val_split)

        model_type = get_model(model['type'], model['params'])
        trained_model = model_type.fit(X_train_new, y_train_new)

        # Predict, evaluate and plot
        predictions = trained_model.predict(X_val)
        rmse = evaluate_model(y_val, predictions)
        plot_predictions(y_val, predictions, model['type'], output_dir, dataset_name="validation")
        
        print(f"Model: {model['type']}, RMSE: {rmse:.4f}")

        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
            best_model_name = model['type']
            best_X_train = X_train
            best_pipeline = pipeline

LinearRegression


TypeError: list indices must be integers or slices, not dict

In [ ]:
import joblib 
import os
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

model = joblib.load("saved_models/best_model.pkl")

Ridge(alpha=10.0, random_state=42)

In [5]:
pipeline = joblib.load("saved_models/preprocessing_pipeline.pkl")
model = joblib.load("saved_models/best_model.pkl")

In [7]:
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ("preprocessing", pipeline),  # your loaded transformer
    ("model", model)              # your trained model
])

In [ ]:
# Save combined artifact
import yaml
def load_config(config_path="configs/shallow4.yaml"):
    with open(config_path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

joblib.dump(full_pipeline, f"{config['output']['saved_models_folder']}/{config['output']['combined_model_filename']}")

['saved_models/combined_model.joblib']

In [3]:
import pandas as pd
import os
import json
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")



# Load CSV
df = pd.read_csv("data/data_raw/df_test.csv")

row = df.iloc[0].to_dict()

# Create the dictionary in the required format
payload = {"features": row}

# Save to JSON file
with open("test_input.json", "w") as f:
    json.dump(payload, f, indent=4)

print("Saved row as JSON in test_input.json")
print(payload)

Saved row as JSON in test_input.json
{'features': {'Unnamed: 0': 8763, 'time': '2018-01-01 00:00:00', 'Madrid_wind_speed': 5.0, 'Valencia_wind_deg': 'level_8', 'Bilbao_rain_1h': 0.0, 'Valencia_wind_speed': 5.0, 'Seville_humidity': 87.0, 'Madrid_humidity': 71.3333333333, 'Bilbao_clouds_all': 20.0, 'Bilbao_wind_speed': 3.0, 'Seville_clouds_all': 0.0, 'Bilbao_wind_deg': 193.3333333333, 'Barcelona_wind_speed': 4.0, 'Barcelona_wind_deg': 176.6666666667, 'Madrid_clouds_all': 0.0, 'Seville_wind_speed': 1.0, 'Barcelona_rain_1h': 0.0, 'Seville_pressure': 'sp25', 'Seville_rain_1h': 0.0, 'Bilbao_snow_3h': 0, 'Barcelona_pressure': 1017.3333333333, 'Seville_rain_3h': 0.0, 'Madrid_rain_1h': 0.0, 'Barcelona_rain_3h': 0.0, 'Valencia_snow_3h': 0, 'Madrid_weather_id': 800.0, 'Barcelona_weather_id': 800.0, 'Bilbao_pressure': 1025.6666666667, 'Seville_weather_id': 800.0, 'Valencia_pressure': nan, 'Seville_temp_max': 284.4833333333, 'Madrid_pressure': 1030.0, 'Valencia_temp_max': 287.4833333333, 'Valencia_

In [ ]:
import requests
import json
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

# Load your test data
with open('test_input.json', 'r') as f:
    test_data = json.load(f)

# Your Cloud Run URL (replace with actual URL)
url = "https://energy-forecast-service-791992711848.europe-west4.run.app"

# Make the request
response = requests.post(url, json=test_data)
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

git add .
git commit -m "Build with secret"
git push

git commit --allow-empty -m "Try build script"
git push


git add .github/workflows/gcp-build-key.yml
git commit -m "Try build script"
git push

https://energy-forecast-service-791992711848.europe-west4.run.app

In [ ]:
import pandas as pd
import os
import json
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")



# Load CSV
df = pd.read_csv("data/data_raw/df_test.csv")

row = df.iloc[0].to_dict()

# Create the dictionary in the required format
payload = {"features": row}

# Save to JSON file
with open("test_input.json", "w") as f:
    json.dump(payload, f, indent=4)

print("Saved row as JSON in test_input.json")
print(payload)

Saved row as JSON in test_input.json
{'features': {'Unnamed: 0': 8763, 'time': '2018-01-01 00:00:00', 'Madrid_wind_speed': 5.0, 'Valencia_wind_deg': 'level_8', 'Bilbao_rain_1h': 0.0, 'Valencia_wind_speed': 5.0, 'Seville_humidity': 87.0, 'Madrid_humidity': 71.3333333333, 'Bilbao_clouds_all': 20.0, 'Bilbao_wind_speed': 3.0, 'Seville_clouds_all': 0.0, 'Bilbao_wind_deg': 193.3333333333, 'Barcelona_wind_speed': 4.0, 'Barcelona_wind_deg': 176.6666666667, 'Madrid_clouds_all': 0.0, 'Seville_wind_speed': 1.0, 'Barcelona_rain_1h': 0.0, 'Seville_pressure': 'sp25', 'Seville_rain_1h': 0.0, 'Bilbao_snow_3h': 0, 'Barcelona_pressure': 1017.3333333333, 'Seville_rain_3h': 0.0, 'Madrid_rain_1h': 0.0, 'Barcelona_rain_3h': 0.0, 'Valencia_snow_3h': 0, 'Madrid_weather_id': 800.0, 'Barcelona_weather_id': 800.0, 'Bilbao_pressure': 1025.6666666667, 'Seville_weather_id': 800.0, 'Valencia_pressure': nan, 'Seville_temp_max': 284.4833333333, 'Madrid_pressure': 1030.0, 'Valencia_temp_max': 287.4833333333, 'Valencia_

In [ ]:
 gsutil iam ch \
  serviceAccount:cloud-run-sa-energy-forecast-i@energy-forecast-467113.iam.gserviceaccount.com:roles/storage.Consumer\
  gs://energy-forecast-467113_cloudbuild

In [6]:
import os
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

import pandas as pd
features = pd.read_csv("saved_models/train_features.csv")

features.head()
print(features.columns)
print(features.shape)

train_df= pd.read_csv("data/data_raw/df_train.csv")
print(train_df.dtypes)

Index(['Unnamed: 0', 'time', 'Madrid_wind_speed', 'Valencia_wind_deg',
       'Bilbao_rain_1h', 'Valencia_wind_speed', 'Seville_humidity',
       'Madrid_humidity', 'Bilbao_clouds_all', 'Bilbao_wind_speed',
       'Seville_clouds_all', 'Bilbao_wind_deg', 'Barcelona_wind_speed',
       'Barcelona_wind_deg', 'Madrid_clouds_all', 'Seville_wind_speed',
       'Barcelona_rain_1h', 'Seville_pressure', 'Seville_rain_1h',
       'Bilbao_snow_3h', 'Barcelona_pressure', 'Seville_rain_3h',
       'Madrid_rain_1h', 'Barcelona_rain_3h', 'Valencia_snow_3h',
       'Madrid_weather_id', 'Barcelona_weather_id', 'Bilbao_pressure',
       'Seville_weather_id', 'Valencia_pressure', 'Seville_temp_max',
       'Madrid_pressure', 'Valencia_temp_max', 'Valencia_temp',
       'Bilbao_weather_id', 'Seville_temp', 'Valencia_humidity',
       'Valencia_temp_min', 'Barcelona_temp_max', 'Madrid_temp_max',
       'Barcelona_temp', 'Bilbao_temp_min', 'Bilbao_temp',
       'Barcelona_temp_min', 'Bilbao_temp_max', 'Sev

In [1]:
import os
os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

import hydra
from omegaconf import OmegaConf, DictConfig
import random

@hydra.main(version_base=None, config_path="configs", config_name="config")
def main(cfg: DictConfig) -> None:
    # Set seed
    print("hi")
    random.seed(cfg.seed)

    # Print full config
    print(OmegaConf.to_yaml(cfg))

    # Access models
    for i, model_cfg in enumerate(cfg.models):
        print(f"Model {i+1}: {model_cfg.type}, scaling={model_cfg.scaling}")

     # Example: use data path
    train_file = f"{cfg.data.data_path}/{cfg.data.train_path}"
    print(f"Loading training data from: {train_file}")

main()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\lackerman008\AppData\Roaming\jupyter\runtime\kernel-v30afaeead31d1ad32fdb37455e5586cb18a285023.json


SystemExit: 2

C:\Users\lackerman008\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
from pathlib import Path

from src.electricity_forecast.config_and_logging import load_config_hydra

os.chdir("c:/Users/lackerman008/OneDrive - pwc/Outside/Code/Machine learning/Electricity Shortfall Challenge")

# Get the project root directory

config_name = "config_cv"

config = load_config_hydra(config_name=config_name)


client = MlflowClient()
model_name = config['logging']['registered_model_name']
versions = client.get_model_version_by_alias(model_name, "production")


In [23]:
print(versions.version)
model_type = versions.tags.get("model_type")
print(model_type)

21
Ridge


In [16]:
print(versions)
print(versions[0])

client.set_registered_model_alias(
    name=config['logging']['registered_model_name'],
    version=17,
    alias="experiment"
)

<ModelVersion: aliases=['production'], creation_timestamp=1754634361865, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1754634361865, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='final_cv_rmse', model_id='m-426666e6f2d1424187f702039e1a18e8', run_id='461f2b32223b48e1af7da6d47a234fd6', step=0, timestamp=1754634342448, value=5025.032922056471>], model_id='m-426666e6f2d1424187f702039e1a18e8', name='energy-forecast-model', params={}, run_id='461f2b32223b48e1af7da6d47a234fd6', run_link=None, source='models:/m-426666e6f2d1424187f702039e1a18e8', status='READY', status_message=None, tags={}, user_id=None, version=19>


TypeError: 'ModelVersion' object is not subscriptable

In [14]:
print(versions)

[<ModelVersion: aliases=[], creation_timestamp=1754583387534, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1754583387534, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='final_cv_rmse', model_id='m-97a8277531b04bb3ad5605c3f038155b', run_id='a2296ee579da4ae9998399dbbe75b4df', step=0, timestamp=1754583368961, value=5007.451417112638>], model_id='m-97a8277531b04bb3ad5605c3f038155b', name='energy-forecast-model', params={}, run_id='a2296ee579da4ae9998399dbbe75b4df', run_link=None, source='models:/m-97a8277531b04bb3ad5605c3f038155b', status='READY', status_message=None, tags={}, user_id=None, version=18>]


In [1]:
param_grid = {
}
print(len(param_grid))

0
